# 🧠 Petunjuk Lengkap Menjalankan Sistem Round Robin Load Balancer (2 Komputer)

Panduan ini menjelaskan langkah demi langkah menjalankan proyek **Round Robin Load Balancer** berbasis **Python (Flask)** menggunakan **dua komputer**:
- 🖥️ **Komputer 1 (Windows)** → Agent-1  
- 💻 **Komputer 2 (Ubuntu/Linux)** → Load Balancer + Agent-2 + Client Tester  

Tujuan sistem ini adalah untuk membagi permintaan dari client secara **bergantian (Round Robin)** antara dua agent dan menganalisis waktu responsnya.


## 🧩 1. Struktur Direktori Proyek

### 💻 Komputer 2 (Ubuntu) – Server Utama
```
roundrobin-server/
│
├── load_balancer.py
├── agent2.py
├── client_load_test.py
├── analyze_log.py
└── lb_log.csv
```

### 🖥️ Komputer 1 (Windows) – Agent 1
```
roundrobin-agent1/
│
└── agent1.py
```


## ⚙️ 2. Persiapan Lingkungan

### A. Di Ubuntu (Server)
```bash
sudo apt update
sudo apt install python3.12 python3.12-venv -y

mkdir ~/roundrobin-server
cd ~/roundrobin-server
python3 -m venv venv
source venv/bin/activate

pip install flask requests pandas matplotlib
```

### B. Di Windows (Agent 1)
```bash
python -m pip install --upgrade pip
python -m pip install flask
```


## 🖥️ 3. Menjalankan Agent-1 (Windows)
Buka Command Prompt dan masuk ke folder proyek:
```bash
cd C:\path\to\roundrobin-agent1
```
Buat file `agent1.py` dengan isi berikut:


In [ ]:
# agent1.py
from flask import Flask, jsonify
import time, socket

app = Flask(__name__)

@app.route('/')
def handle_request():
    start = time.time()
    hostname = socket.gethostname()
    resp_ms = round((time.time() - start) * 1000, 2)
    return jsonify({"agent": hostname, "status": "success", "response_time_ms": resp_ms})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)


Jalankan Agent-1:
```bash
python agent1.py
```
Pastikan muncul:
```
* Running on http://0.0.0.0:5001
```
Uji dari komputer server (Ubuntu):
```
http://192.168.1.70:5001/
```


## 💻 4. Menjalankan Agent-2 (Ubuntu)
Masuk ke folder proyek dan buat file `agent2.py`:


In [ ]:
# agent2.py
from flask import Flask, jsonify
import time, socket

app = Flask(__name__)

@app.route('/')
def handle_request():
    start = time.time()
    hostname = socket.gethostname()
    resp_ms = round((time.time() - start) * 1000, 2)
    return jsonify({"agent": hostname, "status": "success", "response_time_ms": resp_ms})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5002)


Jalankan Agent-2:
```bash
source venv/bin/activate
python agent2.py
```
Akses di browser:
```
http://192.168.1.72:5002/
```


## ⚙️ 5. Menjalankan Load Balancer (Ubuntu)
Buat file `load_balancer.py` di folder `roundrobin-server`:


In [ ]:
# load_balancer.py
from flask import Flask, jsonify
import requests, time, csv

app = Flask(__name__)

agents = ["http://192.168.1.70:5001", "http://192.168.1.72:5002"]
current = 0

@app.route('/request')
def distribute():
    global current
    agent = agents[current]
    current = (current + 1) % len(agents)
    start = time.time()
    try:
        r = requests.get(agent, timeout=3)
        elapsed = round((time.time() - start) * 1000, 2)
        backend = agent.split("//")[1]
        with open("lb_log.csv", "a", newline="") as f:
            csv.writer(f).writerow([time.strftime("%H:%M:%S"), backend, elapsed])
        return jsonify({"backend": backend, "elapsed_ms": elapsed, "response": r.json()})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    with open("lb_log.csv", "w", newline="") as f:
        csv.writer(f).writerow(["timestamp", "backend", "elapsed_ms"])
    app.run(host='0.0.0.0', port=8000)


Jalankan Load Balancer:
```bash
python load_balancer.py
```
Pastikan muncul:
```
* Running on http://0.0.0.0:8000
```
Uji di browser:
```
http://192.168.1.72:8000/request
```


## 🧪 6. Menjalankan Client Tester (Ubuntu)
Buat file `client_load_test.py`:


In [ ]:
# client_load_test.py
import requests, time

def run_test(n):
    url = "http://192.168.1.72:8000/request"
    print(f"\n=== Uji Beban {n} Request ===")
    for i in range(1, n+1):
        start = time.time()
        try:
            r = requests.get(url)
            elapsed = round((time.time() - start) * 1000, 2)
            backend = r.json().get("backend", "Unknown")
            print(f"Req-{i:03d} | {elapsed:8.2f} ms | Backend: {backend}")
        except Exception as e:
            print(f"Req-{i:03d} | ERROR | {e}")
        time.sleep(0.05)

# Jalankan semua skenario
for req in [50, 100, 200]:
    run_test(req)


### 🔹 Menjalankan Hanya 50 Request Saja
Jika kamu hanya ingin menguji **50 request** saja, jalankan perintah berikut:
```bash
python client_load_test.py  # lalu hentikan manual setelah 50 request
```
Atau ubah bagian akhir script menjadi:
```python
run_test(50)
```
Kemudian jalankan:
```bash
python client_load_test.py
```


## 📊 7. Analisis Hasil Log
Buat file `analyze_log.py`:


In [ ]:
# analyze_log.py
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("lb_log.csv")
print("\n📈 Statistik Waktu Respons per Backend:")
print(df.groupby("backend")["elapsed_ms"].describe())

plt.figure(figsize=(8,5))
df.groupby("backend")["elapsed_ms"].mean().plot(kind="bar")
plt.ylabel("Rata-rata Waktu Respons (ms)")
plt.title("Perbandingan Rata-rata Respons Tiap Agent")
plt.show()


### 🧮 Menjalankan Analisis Log
Setelah pengujian selesai, jalankan perintah berikut di Ubuntu:
```bash
python analyze_log.py
```
Output:
- Menampilkan tabel statistik waktu respons per backend.  
- Menampilkan grafik perbandingan rata-rata waktu respons antara Agent-1 dan Agent-2.


## ✅ 8. Hasil Akhir
Jika berhasil:
- Load Balancer akan menyalurkan request secara **bergantian (Round Robin)** ke Agent-1 dan Agent-2.  
- Semua hasil uji disimpan otomatis dalam `lb_log.csv`.  
- File `analyze_log.py` akan menampilkan grafik dan statistik performa.

> 💡 Tips: Pastikan IP address di `load_balancer.py` sesuai dengan jaringan lokal kamu.
